### Our main objective for this code is to find segments that vary in payment amounts/behaviour the assumption being that each has a stable segment level performance, which is a reasonable assumption because we homogeneous cohorts with respect to the segment/model score.

We are working with synthetic data, so there are distributions and such that doesn't follow the norm for real data, despite this - building the scaffolding for creating a CLV will still be pertinent.

Below is a class that loads the synthetic data, and maps the columns CustomerID,TransactionDate,TransactionAmount to the correct name. There are also methods in this class for creating RFM measures, RFM Segments, Demographics and Transaction Descriptor features (which I dont use), if you can do something useful with it then by all means.

Please Note that the dataloader, and rfm methods are general methods they can be used with any data that contains the columns CustomerID,TransactionID,TransactionDate,TransactionAmount.

I will close off by saying that the point of this exercise is to end off with segments that have stable performance at the "crowd" level. Thus nullifying the problem of high variance when trying to do a regression.

Once we have these segments, we have a choice of 2, we will then extrapolate payments per cohort/crowd/group. That will be the next step, this work will be carried out in a seperate notebook and class with written for the Vintage object with optimisation methods.

In [16]:
import pandas as pd
from pathlib import Path
import sklearn
import category_encoders as ce
import xgboost as xgb
import optuna
import matplotlib.pyplot as plt
import numpy as np
import rfmbinner

class DataLoader:
    
    def __init__(self,path,customer_id,transaction_id,transaction_date,amount):
        """
        Initialize the DataLoader.

        Parameters:
            path (str): Path to the CSV file.
            customer_id (str): Column name for customer IDs.
            transactionid (str): Column name for transaction IDs.
            transaction_date (str): Column name for transaction dates.
            amount (str): Column name for transaction amounts.

        """
        self.path = path
        self.customer_id = customer_id
        self.transaction_id = transaction_id
        self.transaction_date = transaction_date 
        self.amount = amount 
    
    def fetch_data(self) -> pd.DataFrame:
        """
        Fetch raw CSV data from the specified path.

        Parameters:
            path (str): Path to the CSV file.

        Returns:
            pd.DataFrame: Loaded data.
        """
        data_path = Path(self.path)
        if not data_path.exists():
            raise FileNotFoundError(f"CSV file not found at: {data_path}")
        data = pd.read_csv(data_path)
        data.rename(columns={self.customer_id:'CustomerID',
                             self.transaction_id: 'TransactionID',
                             self.transaction_date: 'Date',
                             self.amount: 'Amount'}, inplace=True)
        return data
    
    def calculate_rfm(self, snapshot_date: str, window: pd.Timedelta,df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate RFM metrics for customer segmentation.

        Parameters:
            snapshot_date (str): Date to calculate recency from (YYYY-MM-DD).

        Returns:
            pd.DataFrame: RFM metrics per CustomerID.
        """
        data = df.copy()
        snapshot = pd.to_datetime(snapshot_date)
        data['Date'] = pd.to_datetime(data.Date)
        data  = data[(data.Date<snapshot) & (data.Date>(snapshot-window))]
        data['recency'] = (snapshot - pd.to_datetime(data['Date'])).dt.days
        print(data)
        rfm = data.groupby('CustomerID').agg({
            'recency': 'min',
            'TransactionID': 'count',               # frequency: number of transactions
            'Amount': 'sum'             # monetary: total spend
        }).reset_index()

        rfm.rename(columns={'TransactionID': 'frequency', 'Amount': 'monetary'}, inplace=True)
        rfm['date'] = snapshot_date

        return rfm[['CustomerID', 'date', 'recency', 'frequency', 'monetary']]
    
    def calculate_target(self,date: str, window_size: pd.Timedelta, \
                         repurchase_threshold: float, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculating the target for the model
        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
            window_size (pd.Timedelta): Number of days in which to consider repurchase
            repurchase_threshold: Minimum spend to be considered a repurchase
        Returns:
            pd.DataFrame: Target per CustomerID.
        """
        #caculate whether a customer made total purchases after date exceeding repurchase threshold
        data = df.copy()
        data['Date'] = pd.to_datetime(data['Date'])
        date = pd.to_datetime(date)
        future_purchases = data[(data['Date'] > date) & \
                                (data['Date'] <= date + window_size)]

        target_customers = future_purchases.groupby('CustomerID')[['Amount']].sum()
        target_customers = target_customers[target_customers >= repurchase_threshold].index

        total_future_purchases = future_purchases.groupby('CustomerID')[['Amount']].sum()
        data = data.merge(total_future_purchases.rename(columns={'Amount': 'subsequent_purchases'}),
                            on='CustomerID', how='left')
        data['subsequent_purchases'] = data['subsequent_purchases'].fillna(0)


        data['target'] = data['CustomerID'].isin(target_customers).astype(int)
        data['date'] = date
        return data[['CustomerID','date','target','subsequent_purchases']].drop_duplicates()
    
    def rfm_segments(self,date: str, window: pd.Timedelta, df: pd.DataFrame) -> pd.DataFrame:
        """
        Segments customers based on their RFM score.

        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
            df (pd.DataFrame): Raw transactional data.

        Returns:
            pd.DataFrame: RFM segments per CustomerID.
        """
        data = df.copy()
        data = self.calculate_rfm(date,window,data)
        # Create RFM scores
        data['r_score'] = pd.qcut(data['recency'].rank(method='first'), 5, labels=[5, 4, 3, 2, 1])
        data['f_score'] = pd.qcut(data['frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
        data['m_score'] = pd.qcut(data['monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
        data['rfm_score'] = data['r_score'].astype(str) + \
                            data['f_score'].astype(str) + \
                            data['m_score'].astype(str)
        data['rfm_score_int'] = data['r_score'].astype(int) + \
                            data['f_score'].astype(int)+ \
                            data['m_score'].astype(int)           
        #create a dicitionary with the rfm_score that maps into segment, the key must be a regex
        segment_map = {
                    r'^55[1-5]$': 'Champions',
                    r'^54[1-5]$': 'Loyal Customers',
                    r'^45[1-5]$': 'Potential Loyalists',
                    r'^53[1-5]$': 'New or Returning Customers',
                    r'^33[1-5]$': 'Promising',
                    r'^22[1-5]$': 'Needs Attention',
                    r'^\d{3}$': 'Others'  # Matches any other 3-digit score
                }
        data['segment'] = data['rfm_score'].replace(segment_map, regex=True)
        data['date'] = date
        return data[['CustomerID', 'date', 'rfm_score','rfm_score_int', 'segment']]
    
    def dedup_demographic_variables(self,df: pd.DataFrame) -> pd.DataFrame:
        """
        Deduplicate demographic variables for each customer, keeping the earliest entry.

        Parameters:
            df (pd.DataFrame): Raw transactional data with demographic information.

        Returns:
            pd.DataFrame: Deduplicated demographic data per CustomerID.
        """
        data = df.copy()
        data['Date'] = pd.to_datetime(data['Date'])
        
        # Sort by CustomerID and Date to get the most recent demographic info
        data = data.sort_values(by=['CustomerID', 'Date'], ascending=True)
        
        # Drop duplicates, keeping the last (most recent) entry for each CustomerID
        # Assuming demographic variables are 'Gender', 'Age', 'Age Group', extend to the actual list
        demographic_cols = ['CustomerID', 'Gender', 'Age','Province']
        deduplicated_demographics = data[demographic_cols].drop_duplicates(subset=['CustomerID'], keep='first')
        
        return deduplicated_demographics
    
    def transaction_descriptor_variables(self,date) -> pd.DataFrame:

        data = self.fetch_data()
        data['Date'] = pd.to_datetime(data['Date'])
        purchases = data[data.Date < date]
        summary = purchases.groupby('CustomerID')[['ProductCategory','PurchaseChannel',
                                       'PaymentMethod','Store']].agg(pd.Series.mode).reset_index()
        summary['date'] = date
        # Rename columns for clarity
        summary.rename(columns={'ProductCategory': 'Most_frequented_Category',
                                  'PurchaseChannel': 'Most_frequented_Channel',
                                  'PaymentMethod': 'Most_used_payment_method',
                                  'Store': 'Most_frequented_Store'}, inplace=True)
        return summary[['CustomerID','date','Most_frequented_Channel','Most_frequented_Category','Most_used_payment_method','Most_frequented_Store']]
    
    def dedup_demographic_variables(self,df: pd.DataFrame) -> pd.DataFrame:
        """
        Deduplicate demographic variables for each customer, keeping the latest entry.

        Parameters:
            df (pd.DataFrame): Raw transactional data with demographic information.

        Returns:
            pd.DataFrame: Deduplicated demographic data per CustomerID.
        """
        data = df.copy()
        data['Date'] = pd.to_datetime(data['Date'])
        
        # Sort by CustomerID and Date to get the most recent demographic info
        data = data.sort_values(by=['CustomerID', 'Date'], ascending=True)
        
        # Drop duplicates, keeping the last (most recent) entry for each CustomerID
        # Assuming demographic variables are 'Gender', 'Age', 'Age Group', extend to the actual list
        demographic_cols = ['CustomerID', 'Gender', 'Age','Province']
        deduplicated_demographics = data[demographic_cols].drop_duplicates(subset=['CustomerID'], keep='last')
        
        return deduplicated_demographics
    


For the code below the process is as follows:

1. Initialise a dataloader
2. Fetch the synthetic data
3. Run the appropriate methods of the dataloader in order to create a collection of features
4. We join all the features together and create a "model_data" dataframe.

In [ ]:
params = {'path':r'data\synthetic_transactions.csv',
            'customer_id':'CustomerID',
            'transaction_id':'TransactionID',
            'transaction_date':'TransactionDate',
            'amount':'TransactionAmount'}
loader = DataLoader(**params)
data = loader.fetch_data()

In [18]:
start_date = '2022-08-30'
end_date = pd.to_datetime('2025-08-07')
pivot_date = pd.to_datetime(start_date)
rfms = []
targets = []
rfm_segments = []
transaction_descriptor_data = []

while pivot_date<end_date:
    rfms.append(loader.calculate_rfm(pivot_date,pd.Timedelta(weeks=4),data))
    rfm_segments.append(loader.rfm_segments(pivot_date,pd.Timedelta(weeks=4),data))
    targets.append(loader.calculate_target(pivot_date,pd.Timedelta(weeks=12),0,data))
    pivot_date+=pd.Timedelta(weeks=4)

rfm_data = pd.concat(rfms)
target_data = pd.concat(targets)
rfm_segments = pd.concat(rfm_segments)
dem_data = loader.dedup_demographic_variables(data)

       CustomerID  TransactionID                Date   Amount  \
44     CUST-06281  TRANS-0000045 2022-08-22 11:40:14   149.73   
103    CUST-02316  TRANS-0000104 2022-08-10 09:23:20   355.22   
144    CUST-00782  TRANS-0000145 2022-08-26 07:43:02   167.62   
256    CUST-09787  TRANS-0000257 2022-08-14 23:07:00  1375.65   
344    CUST-03613  TRANS-0000345 2022-08-11 06:01:42   254.17   
...           ...            ...                 ...      ...   
99595  CUST-06341  TRANS-0099596 2022-08-23 11:42:46   541.92   
99713  CUST-02059  TRANS-0099714 2022-08-27 02:41:30    20.31   
99732  CUST-06166  TRANS-0099733 2022-08-23 03:26:52   322.08   
99779  CUST-08342  TRANS-0099780 2022-08-25 11:31:46    80.34   
99963  CUST-03814  TRANS-0099964 2022-08-20 04:54:33   337.43   

       ProductCategory  Age       Province  Gender  PaymentMethod  \
44     Health & Beauty   35     Mpumalanga    Male            EFT   
103      Home & Garden   59     North West    Male            EFT   
144         

In [19]:
model_data = rfm_data.merge(target_data,how='left',left_on=['CustomerID','date'],right_on=['CustomerID','date'])
model_data = model_data.merge(dem_data,how='left',left_on='CustomerID',right_on='CustomerID')
#model_data = model_data.merge(transaction_descriptor_data,how='left', left_on=['CustomerID','date'],right_on=['CustomerID','date'])
model_data = model_data.merge(rfm_segments,how='left', left_on=['CustomerID','date'],right_on=['CustomerID','date'])

In [20]:
end_date_of_training = '2024-07-31'
model_data = model_data.set_index(['CustomerID','date']).drop('rfm_score',axis=1)
model_data = model_data[model_data.index.get_level_values(1)<end_date_of_training]

In [21]:
model_data.target.value_counts()/len(model_data)

target
1    0.533847
0    0.466153
Name: count, dtype: float64

In [22]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    model_data.drop(['target', 'subsequent_purchases'], axis=1),
    model_data['target'],
    test_size=0.2,
    random_state=42
)

for col in X_train.columns:
    sample = X_train[col].iloc[0]
    if isinstance(sample, (list, np.ndarray)):
        print(f"Column '{col}' contains unhashable types: {type(sample)}")

encoder = ce.TargetEncoder()
X_train = encoder.fit_transform(X_train, y_train)
X_test = encoder.transform(X_test)

# Optuna + XGBoost training
def train_xgboost_with_optuna(X, y, n_trials=50, test_size=0.2, random_state=42):
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    def objective(trial):
        params = {
            "verbosity": 0,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "eta": trial.suggest_float("eta", 0.01, 0.3),
        }

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        model = xgb.train(params, dtrain, num_boost_round=100,
                          evals=[(dval, "validation")],
                          early_stopping_rounds=10,
                          verbose_eval=False)
        preds = model.predict(dval)
        return sklearn.metrics.roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    best_params.update({
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "logloss"
    })

    final_model = xgb.train(best_params, xgb.DMatrix(X, label=y), num_boost_round=study.best_trial.number)
    return final_model, study

# Train and evaluate
model, study = train_xgboost_with_optuna(X_train, y_train)
y_probs = model.predict(xgb.DMatrix(X_test))

print(sklearn.metrics.classification_report(y_test, y_probs > 0.5))
print("ROC AUC:", sklearn.metrics.roc_auc_score(y_test, y_probs))


[I 2025-08-09 10:27:13,383] A new study created in memory with name: no-name-6aa18572-67f0-4c37-8dbd-466c3edd94eb
[I 2025-08-09 10:27:13,449] Trial 0 finished with value: 0.49855258905484423 and parameters: {'booster': 'gbtree', 'lambda': 9.708761089817045, 'alpha': 0.14164066650555962, 'subsample': 0.927034541457574, 'colsample_bytree': 0.6759997778049874, 'max_depth': 6, 'eta': 0.13883847573465777}. Best is trial 0 with value: 0.49855258905484423.
[I 2025-08-09 10:27:13,517] Trial 1 finished with value: 0.49906834602742134 and parameters: {'booster': 'dart', 'lambda': 0.019269313890203827, 'alpha': 0.0009530920300415258, 'subsample': 0.560399031880777, 'colsample_bytree': 0.5616808284467161, 'max_depth': 10, 'eta': 0.07580761047584303}. Best is trial 1 with value: 0.49906834602742134.
[I 2025-08-09 10:27:13,558] Trial 2 finished with value: 0.4937267918673293 and parameters: {'booster': 'gbtree', 'lambda': 0.006503091761647164, 'alpha': 4.00173716677753e-05, 'subsample': 0.8446667786

              precision    recall  f1-score   support

           0       0.47      0.18      0.26      5483
           1       0.53      0.81      0.64      6143

    accuracy                           0.52     11626
   macro avg       0.50      0.50      0.45     11626
weighted avg       0.50      0.52      0.46     11626

ROC AUC: 0.5002403949709859


In [23]:
sklearn.metrics.roc_auc_score(y_test,model.predict(xgb.DMatrix(X_test)))

0.5002403949709859

In [24]:
X_test['QSegment'] = pd.qcut(y_probs,10)

In [25]:
summary_data = X_test.merge(target_data,how='left', left_on='CustomerID', right_on='CustomerID')
summary = summary_data.groupby(['QSegment']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})
summary = summary.rename(columns = {'recency':'Count','monetary':'prior_purchases','target':'Purchase Rate'})
summary['prior_purchases in ZAR'] = summary.prior_purchases.apply(lambda x: f'R {x:,.2f}')
summary['Value %'] = (summary['prior_purchases']/summary['prior_purchases'].sum()*100).apply(lambda x: f'{x:.2f}%')
summary['AVG subs. spend in ZAR'] = (summary['subsequent_purchases']/summary.Count).apply(lambda x: f'R {x:.2f}')
summary['subsequent_purchases in ZAR (3 month window)'] = summary.subsequent_purchases.apply(lambda x: f"R {x:,.2f}")
summary['Purchase Rate'] =  (summary['Purchase Rate']/summary.Count).apply( lambda x: f'{x:.2%}')
summary.drop(['prior_purchases','subsequent_purchases'],axis=1)

C:\Users\wesle\AppData\Local\Temp\ipykernel_3156\2678178072.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = summary_data.groupby(['QSegment']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})


,Count,Purchase Rate,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
QSegment,,,,,,
"(0.267, 0.482]",45357,54.82%,"R 31,220,993.31",12.16%,R 397.48,"R 18,028,312.36"
"(0.482, 0.502]",45357,54.08%,"R 24,744,991.44",9.64%,R 384.60,"R 17,444,470.18"
"(0.502, 0.516]",45318,55.08%,"R 25,391,736.63",9.89%,R 395.31,"R 17,914,488.00"
"(0.516, 0.526]",45357,54.41%,"R 24,366,821.31",9.49%,R 390.18,"R 17,697,575.52"
"(0.526, 0.535]",45318,54.00%,"R 24,114,681.63",9.39%,R 392.01,"R 17,765,007.38"
"(0.535, 0.545]",45357,54.82%,"R 25,126,569.00",9.78%,R 400.62,"R 18,171,006.39"
"(0.545, 0.555]",45318,54.32%,"R 23,599,423.38",9.19%,R 392.47,"R 17,785,746.02"
"(0.555, 0.568]",45357,54.87%,"R 24,307,279.62",9.47%,R 399.16,"R 18,104,809.97"
"(0.568, 0.589]",45318,55.32%,"R 24,492,518.31",9.54%,R 401.69,"R 18,203,934.37"


In [26]:
summary_data = X_test.merge(target_data,how='left', left_on='CustomerID', right_on='CustomerID')
summary = summary_data.groupby(['rfm_score_int']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})
summary = summary.rename(columns = {'recency':'Count','monetary':'prior_purchases','target':'Purchase Rate'})
summary['prior_purchases in ZAR'] = summary.prior_purchases.apply(lambda x: f'R {x:,.2f}')
summary['Value %'] = (summary['prior_purchases']/summary['prior_purchases'].sum()*100).apply(lambda x: f'{x:.2f}%')
summary['AVG subs. spend in ZAR'] = (summary['subsequent_purchases']/summary.Count).apply(lambda x: f'R {x:.2f}')
summary['subsequent_purchases in ZAR (3 month window)'] = summary.subsequent_purchases.apply(lambda x: f"R {x:,.2f}")
summary['Purchase Rate'] =  (summary['Purchase Rate']/summary.Count).apply( lambda x: f'{x:.2%}')
summary.drop(['prior_purchases','subsequent_purchases'],axis=1)

,Count,Purchase Rate,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
rfm_score_int,,,,,,
3,4407,55.16%,"R 259,149.15",0.10%,R 363.33,"R 1,601,180.12"
4,12168,55.94%,"R 1,292,137.86",0.50%,R 379.50,"R 4,617,741.59"
5,25506,53.98%,"R 4,429,308.00",1.72%,R 373.17,"R 9,517,950.20"
6,41340,55.15%,"R 9,526,930.92",3.71%,R 374.96,"R 15,500,944.11"
7,54756,54.95%,"R 17,550,243.75",6.83%,R 380.88,"R 20,855,289.45"
8,65325,54.15%,"R 27,251,154.84",10.61%,R 374.20,"R 24,444,491.11"
9,63102,54.44%,"R 33,126,096.51",12.90%,R 396.50,"R 25,020,191.76"
10,56199,55.23%,"R 34,950,472.83",13.61%,R 408.04,"R 22,931,250.80"
11,45318,54.70%,"R 34,011,619.98",13.24%,R 411.30,"R 18,639,212.22"


In [27]:
pd.DataFrame(data = zip(X_train.columns,model.get_score(importance_type='gain').values()),columns=['Features','Importances'])

,Features,Importances
0,recency,2.147652
1,frequency,1.657698
2,monetary,2.672148
3,Gender,1.473629
4,Age,2.364671
5,Province,1.714745
6,rfm_score_int,1.712199
7,segment,1.617197


In [28]:
model_data.segment.unique()

array(['Others', 'Champions', 'Potential Loyalists', 'Needs Attention',
       'New or Returning Customers', 'Promising', 'Loyal Customers'],
      dtype=object)

In [34]:
summary

,Count,prior_purchases,Purchase Rate,subsequent_purchases,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
rfm_score_int,,,,,,,,
3,4407,259149.15,55.16%,1601180.12,"R 259,149.15",0.10%,R 363.33,"R 1,601,180.12"
4,12168,1292137.86,55.94%,4617741.59,"R 1,292,137.86",0.50%,R 379.50,"R 4,617,741.59"
5,25506,4429308.00,53.98%,9517950.20,"R 4,429,308.00",1.72%,R 373.17,"R 9,517,950.20"
6,41340,9526930.92,55.15%,15500944.11,"R 9,526,930.92",3.71%,R 374.96,"R 15,500,944.11"
7,54756,17550243.75,54.95%,20855289.45,"R 17,550,243.75",6.83%,R 380.88,"R 20,855,289.45"
8,65325,27251154.84,54.15%,24444491.11,"R 27,251,154.84",10.61%,R 374.20,"R 24,444,491.11"
9,63102,33126096.51,54.44%,25020191.76,"R 33,126,096.51",12.90%,R 396.50,"R 25,020,191.76"
10,56199,34950472.83,55.23%,22931250.80,"R 34,950,472.83",13.61%,R 408.04,"R 22,931,250.80"
11,45318,34011619.98,54.70%,18639212.22,"R 34,011,619.98",13.24%,R 411.30,"R 18,639,212.22"
